# Branching ratios
Checking the branching ratios using the Meijer Hamiltonian

## Setting up

In [1]:
import numpy as np
import pickle
import sys
import sympy
from tqdm.notebook import tqdm

sys.path.append('../B-state-classes-and-functions/')

from classes import CoupledBasisState, UncoupledBasisState, State
from functions import matrix_to_states, vector_to_state, find_state_idx_from_state, find_closest_state

from sympy.physics.wigner import wigner_3j, wigner_6j

def threej_f(j1,j2,j3,m1,m2,m3):
    return complex(wigner_3j(j1,j2,j3,m1,m2,m3))

def sixj_f(j1,j2,j3,j4,j5,j6):
    return complex(wigner_6j(j1,j2,j3,j4,j5,j6))


with open("B_hamiltonians_symbolic_coupled_mF_is_0.py",'rb') as f:
    hamiltonians_B = pickle.load(f)
    
with open("TlF_X_state_hamiltonian_mF0.py",'rb') as f:
    hamiltonians_X = pickle.load(f)



def ni_range(x0, x1, dx=1):
    # sanity check arguments
    if dx==0:
        raise ValueError("invalid parameters: dx==0")
    if x0>x1 and dx>=0:
        raise ValueError("invalid parameters: x0>x1 and dx>=0")
    if x0<x1 and dx<=0:
        raise ValueError("invalid parameters: x0<x1 and dx<=0")
        
    # generate range list
    range_list = []
    x = x0
    while x < x1:
        range_list.append(x)
        x += dx
    return range_list

#Generate quantum numbers
I_Tl = 1/2             # I1 in Ramsey's notation
I_F  = 1/2             # I2 in Ramsey's notation

#Generate list of ground states
Jmin_g = 0
Jmax_g = 10 # max J value in Hamiltonian

QN_g = []

for J in np.arange(Jmin_g, Jmax_g+1):
    for m1 in np.arange(-I_Tl,I_Tl+1):
        for m2 in np.arange(-I_F,I_F+1):
            mJ = 0-m1-m2
            if np.abs(mJ) <= J:
                QN_g.append(UncoupledBasisState(J,mJ,I_Tl,m1,I_F,m2,0))
                
Jmin_g = 0
Jmax_g = 10 # max J value in Hamiltonian

QN_g_coupled = [CoupledBasisState(F,0,F1,J,I_F,I_Tl,0)
      for J  in ni_range(Jmin_g, Jmax_g+1)
      for F1 in ni_range(np.abs(J-I_F),J+I_F+1)
      for F in ni_range(np.abs(F1-I_Tl),F1+I_Tl+1)
     ]
                
#Generate list of excited states
Jmin_e = 1
Jmax_e = 4

QN_e = []

Omegas = [-1, 1]
QN_e = [CoupledBasisState(F,0,F1,J,I_F,I_Tl,Omega)
      for J  in ni_range(Jmin_e, Jmax_e+1)
      for F1 in ni_range(np.abs(J-I_F),J+I_F+1)
      for F in ni_range(np.abs(F1-I_Tl),F1+I_Tl+1)
      for Omega in Omegas
     ]


### Eigenstates for B

In [2]:
variables = [
    *sympy.symbols('Brot Drot H_const'),
    *sympy.symbols('h1_Tl h1_F'),
    sympy.symbols('q'),
    sympy.symbols('c_Tl'),
    sympy.symbols('c1p_Tl'),
    *sympy.symbols('gL gS')
]

lambdified_hamiltonians_B = {
    H_name : sympy.lambdify(variables, H_matrix)
    for H_name, H_matrix in hamiltonians_B.items()
}

Insert the numerical constants:

In [3]:
#Constants in MHz
Brot = 6687.879e6
Drot = 0.010869e6
H_const = -8.1e-2
h1_Tl = 28789e6
h1_F = 861e6
q = 2.423e6
c_Tl = -7.83e6
c_Tl_p = 11.17e6
gL = 1
gS = 2

H = {
    H_name : H_fn(
        Brot, Drot, H_const,
        h1_Tl, h1_F,
        q,
        c_Tl,
        c_Tl_p,
        gL, gS
    )
    for H_name, H_fn in lambdified_hamiltonians_B.items()
}



In [4]:
Hff_e = H["Hrot"]+H["H_mhf_Tl"]+H["H_mhf_F"]+H["H_c_Tl"]+H["H_LD"]+H["H_cp1_Tl"]
print(np.allclose(Hff_e, Hff_e.conj().T))
D_e, V_e = np.linalg.eigh(Hff_e)

True


In [5]:
excited_states = matrix_to_states(V_e,QN_e)

In [6]:
for state in excited_states[:8]:
    print("")
    state.print_state(tol=0.01)


+0.7071+0.0000j x |J = 1, F1 = 1/2, F = 0, mF = 0, I1 = 1/2, I2 = 1/2, Omega = -1, P = None>
-0.7071+0.0000j x |J = 1, F1 = 1/2, F = 0, mF = 0, I1 = 1/2, I2 = 1/2, Omega = 1, P = None>

+0.7071+0.0000j x |J = 1, F1 = 1/2, F = 0, mF = 0, I1 = 1/2, I2 = 1/2, Omega = -1, P = None>
+0.7071+0.0000j x |J = 1, F1 = 1/2, F = 0, mF = 0, I1 = 1/2, I2 = 1/2, Omega = 1, P = None>

+0.7069+0.0000j x |J = 1, F1 = 1/2, F = 1, mF = 0, I1 = 1/2, I2 = 1/2, Omega = -1, P = None>
-0.7069+0.0000j x |J = 1, F1 = 1/2, F = 1, mF = 0, I1 = 1/2, I2 = 1/2, Omega = 1, P = None>
+0.0142+0.0000j x |J = 1, F1 = 3/2, F = 1, mF = 0, I1 = 1/2, I2 = 1/2, Omega = -1, P = None>
-0.0142+0.0000j x |J = 1, F1 = 3/2, F = 1, mF = 0, I1 = 1/2, I2 = 1/2, Omega = 1, P = None>
-0.0126+0.0000j x |J = 2, F1 = 3/2, F = 1, mF = 0, I1 = 1/2, I2 = 1/2, Omega = -1, P = None>
-0.0126+0.0000j x |J = 2, F1 = 3/2, F = 1, mF = 0, I1 = 1/2, I2 = 1/2, Omega = 1, P = None>

-0.7069+0.0000j x |J = 1, F1 = 1/2, F = 1, mF = 0, I1 = 1/2, I2 = 1/2, 

### Eigenstates for X

In [7]:
#Substitute values into hamiltonian
variables = [
    sympy.symbols('Brot'),
    *sympy.symbols('c1 c2 c3 c4'),
    sympy.symbols('D_TlF'),
    *sympy.symbols('mu_J mu_Tl mu_F')
]

lambdified_hamiltonians_X = {
    H_name : sympy.lambdify(variables, H_matrix)
    for H_name, H_matrix in hamiltonians_X.items()
}

#Molecular constants

# Values for rotational constant are from "Microwave Spectral tables: Diatomic molecules" by Lovas & Tiemann (1974). 
# Note that Brot differs from the one given by Ramsey by about 30 MHz.
B_e = 6.689873e9
c1 = 126030.0 
c2 = 17890.0
c3 = 700.0
c4 = -13300.0
alpha = 45.0843e6
Brot = B_e - alpha/2
D_TlF = 4.2282 * 0.393430307 *5.291772e-9/4.135667e-15 # [Hz/(V/cm)]
mu_J = 35 # Hz/G
mu_Tl = 1240.5 #Hz/G
mu_F = 2003.63 #Hz/G

H_g = {
    H_name : H_fn(
        Brot,
        c1, c2, c3, c4,
        D_TlF,
        mu_J, mu_Tl, mu_F
    )
    for H_name, H_fn in lambdified_hamiltonians_X.items()
    }

In [8]:
Hff_g = H_g["Hff"]
print(np.allclose(Hff_g, Hff_g.conj().T))
D_g, V_g = np.linalg.eigh(Hff_g)

True


In [9]:
#Transform ground state eigenstates to coupled basis
ground_states = matrix_to_states(V_g,QN_g)
for i, ground_state in enumerate(tqdm(ground_states)):
    ground_states[i] = ground_state.remove_small_components(tol = 0.0001).transform_to_coupled()

In [10]:
for state in ground_states[:8]:
    print("")
    state.print_state(tol=0.01)


-1.0000+0.0000j x |J = 0, F1 = 1/2, F = 1, mF = 0, I1 = 1/2, I2 = 1/2, Omega = 0, P = None>

+1.0000+0.0000j x |J = 0, F1 = 1/2, F = 0, mF = 0, I1 = 1/2, I2 = 1/2, Omega = 0, P = None>

+1.0000+0.0000j x |J = 1, F1 = 1/2, F = 0, mF = 0, I1 = 1/2, I2 = 1/2, Omega = 0, P = None>

+0.9961+0.0000j x |J = 1, F1 = 1/2, F = 1, mF = 0, I1 = 1/2, I2 = 1/2, Omega = 0, P = None>
+0.0886+0.0000j x |J = 1, F1 = 3/2, F = 1, mF = 0, I1 = 1/2, I2 = 1/2, Omega = 0, P = None>

+0.0886+0.0000j x |J = 1, F1 = 1/2, F = 1, mF = 0, I1 = 1/2, I2 = 1/2, Omega = 0, P = None>
-0.9961+0.0000j x |J = 1, F1 = 3/2, F = 1, mF = 0, I1 = 1/2, I2 = 1/2, Omega = 0, P = None>

+1.0000+0.0000j x |J = 1, F1 = 3/2, F = 2, mF = 0, I1 = 1/2, I2 = 1/2, Omega = 0, P = None>

+1.0000+0.0000j x |J = 2, F1 = 3/2, F = 1, mF = 0, I1 = 1/2, I2 = 1/2, Omega = 0, P = None>

+0.9983+0.0000j x |J = 2, F1 = 3/2, F = 2, mF = 0, I1 = 1/2, I2 = 1/2, Omega = 0, P = None>
+0.0580+0.0000j x |J = 2, F1 = 5/2, F = 2, mF = 0, I1 = 1/2, I2 = 1/2, O

## Electric dipole matrix elements
Calculate the electric dipole matrix elements between uncoupled basis states in X and B state 

In [11]:
def ED_ME(ground_state, excited_state, pol_vec = np.array([0,0,1]), rme_only = True):
    """
    Function for calculating electric dipole matrix elements between UncoupledBasisStates.
    
    inputs:
    ground_state = UncoupledBasisState object in the X-state of TlF
    excited_state = UncoupledBasisState object in the B-state of TlF
    pol_vec = polarization vector for the light that is driving the transition
    rme_only = True if want to return reduced matrix element, False if want angular component also
    
    returns:
    ME = (reduced) electric dipole matrix element between ground_state and excited_state
    """
    
    #Find quantum numbers for ground state
    F = ground_state.F
    mF = ground_state.mF
    J = ground_state.J
    F1 = ground_state.F1
    I1 = ground_state.I1
    I2 = ground_state.I2
    Omega_g = ground_state.Omega
    
    #Find quantum numbers for excited state
    Fp = excited_state.F
    mFp = excited_state.mF
    Jp = excited_state.J
    F1p = excited_state.F1
    I1p = excited_state.I1
    I2p = excited_state.I2
    Omega_e = excited_state.Omega
    
    #Calculate the reduced matrix element
    q = Omega_g - Omega_e
    ME = ((-1)**(F1+J+Fp+F1p) * np.sqrt((2*F+1)*(2*Fp+1)*(2*F1p+1)*(2*F1+1)) * sixj_f(F1p,Fp,I2,F,F1,1) 
          * sixj_f(Jp,F1p,I1,F1,J,1) * (-1)**(J-Omega_g) *np.sqrt((2*J+1)*(2*Jp+1)) * threej_f(J,1,Jp,-Omega_g, q, Omega_e)
          * float(np.abs(q) < 2))
    
    #If we want the complete matrix element, calculate angular part
    if not rme_only:
        
        #Calculate elements of the polarization vector in spherical basis
        p_vec = {}
        p_vec[-1] = -1/np.sqrt(2) * (pol_vec[0] + 1j *pol_vec[1])
        p_vec[0] = pol_vec[2]
        p_vec[1] = +1/np.sqrt(2) * (pol_vec[0] - 1j *pol_vec[1])
        
        #Calculate the value of p that connects the states
        p = mF-mFp
        p = p*int(np.abs(p) <= 1)
        #Multiply RME by the angular part
        ME = ME * (-1)**(F-mF) * threej_f(F,1,Fp, -mF, p, mFp) * int(np.abs(p) <= 1)
    
    #Return the matrix element
    return ME
    
    
    

In [12]:
#Generate a matrix that contains the reduced electric dipole matrix elements between all excited and ground states of interest             
def generate_RME_matrix(H, QN_g=QN_g_coupled, QN_e = QN_e):
    #result = sympy.zeros(len(ground_states),len(excited_states))
    result = np.zeros((len(QN_g),len(QN_e)),dtype = complex)
    for i,a in tqdm(enumerate(QN_g)):
        for j,b in enumerate(QN_e):
            result[i,j] = H(a,b)
            
    return result

RMEs = generate_RME_matrix(ED_ME)
RMEs_states = V_g.conj().T @ RMEs @ V_e

def calculate_branching_ratio(ground_state, excited_state, RMEs = RMEs, QN_g = QN_g_coupled, 
                              QN_e = QN_e):
    
    #Get the ground_state and excited_state state vectors
    vec_g = ground_state.state_vector(QN_g)
    vec_e = excited_state.state_vector(QN_e)
    
    #Find the reduced matrix element between ground_state and excited_state (for calculating partial width to ground_state)
    RME_ge = vec_g.conj().T @ RMEs @ vec_e
    
    #Find the reduced matrix elements between the excited_state and all possible ground states
    #so the total width of the excited state can be calculated
    RME_e_all = V_g.conj().T @ RMEs @ vec_e
    
    #Calculate branching ratio = partial width/total width
    branching_ratio = RME_ge*RME_ge.conj()/(RME_e_all.conj().T @ RME_e_all)
        
    return branching_ratio
    


In [13]:
#Loop over all excited states and calculate their branching ratios to all ground states
for excited_state in excited_states:
    print(10*"*")
    print("|e> =")
    excited_state.print_state()
    print("\nBranching ratios:")
    #Loop over ground states
    for ground_state in ground_states:
        br = np.real(calculate_branching_ratio(ground_state, excited_state))
        if br > 0.00005:
            print("{:5f} to \n|g> =".format(br))
            ground_state.print_state(tol = 0.01)
            print("")
        

**********
|e> =
+0.7071+0.0000j x |J = 1, F1 = 1/2, F = 0, mF = 0, I1 = 1/2, I2 = 1/2, Omega = -1, P = None>
-0.7071+0.0000j x |J = 1, F1 = 1/2, F = 0, mF = 0, I1 = 1/2, I2 = 1/2, Omega = 1, P = None>

Branching ratios:
0.747254 to 
|g> =
+0.9961+0.0000j x |J = 1, F1 = 1/2, F = 1, mF = 0, I1 = 1/2, I2 = 1/2, Omega = 0, P = None>
+0.0886+0.0000j x |J = 1, F1 = 3/2, F = 1, mF = 0, I1 = 1/2, I2 = 1/2, Omega = 0, P = None>

0.252746 to 
|g> =
+0.0886+0.0000j x |J = 1, F1 = 1/2, F = 1, mF = 0, I1 = 1/2, I2 = 1/2, Omega = 0, P = None>
-0.9961+0.0000j x |J = 1, F1 = 3/2, F = 1, mF = 0, I1 = 1/2, I2 = 1/2, Omega = 0, P = None>

**********
|e> =
+0.7071+0.0000j x |J = 1, F1 = 1/2, F = 0, mF = 0, I1 = 1/2, I2 = 1/2, Omega = -1, P = None>
+0.7071+0.0000j x |J = 1, F1 = 1/2, F = 0, mF = 0, I1 = 1/2, I2 = 1/2, Omega = 1, P = None>

Branching ratios:
0.666667 to 
|g> =
-1.0000+0.0000j x |J = 0, F1 = 1/2, F = 1, mF = 0, I1 = 1/2, I2 = 1/2, Omega = 0, P = None>

0.333333 to 
|g> =
+1.0000+0.0000j x |

**********
|e> =
+0.6615+0.0000j x |J = 2, F1 = 5/2, F = 2, mF = 0, I1 = 1/2, I2 = 1/2, Omega = -1, P = None>
-0.6615+0.0000j x |J = 2, F1 = 5/2, F = 2, mF = 0, I1 = 1/2, I2 = 1/2, Omega = 1, P = None>
-0.2496+0.0000j x |J = 3, F1 = 5/2, F = 2, mF = 0, I1 = 1/2, I2 = 1/2, Omega = -1, P = None>
-0.2496+0.0000j x |J = 3, F1 = 5/2, F = 2, mF = 0, I1 = 1/2, I2 = 1/2, Omega = 1, P = None>

Branching ratios:
0.000068 to 
|g> =
-1.0000+0.0000j x |J = 0, F1 = 1/2, F = 1, mF = 0, I1 = 1/2, I2 = 1/2, Omega = 0, P = None>

0.000339 to 
|g> =
+1.0000+0.0000j x |J = 2, F1 = 3/2, F = 1, mF = 0, I1 = 1/2, I2 = 1/2, Omega = 0, P = None>

0.002569 to 
|g> =
+0.9983+0.0000j x |J = 2, F1 = 3/2, F = 2, mF = 0, I1 = 1/2, I2 = 1/2, Omega = 0, P = None>
+0.0580+0.0000j x |J = 2, F1 = 5/2, F = 2, mF = 0, I1 = 1/2, I2 = 1/2, Omega = 0, P = None>

0.885013 to 
|g> =
+0.0580+0.0000j x |J = 2, F1 = 3/2, F = 2, mF = 0, I1 = 1/2, I2 = 1/2, Omega = 0, P = None>
-0.9983+0.0000j x |J = 2, F1 = 5/2, F = 2, mF = 0, I1 =

**********
|e> =
+0.6816+0.0000j x |J = 3, F1 = 7/2, F = 3, mF = 0, I1 = 1/2, I2 = 1/2, Omega = -1, P = None>
-0.6816+0.0000j x |J = 3, F1 = 7/2, F = 3, mF = 0, I1 = 1/2, I2 = 1/2, Omega = 1, P = None>
-0.1881+0.0000j x |J = 4, F1 = 7/2, F = 3, mF = 0, I1 = 1/2, I2 = 1/2, Omega = -1, P = None>
-0.1881+0.0000j x |J = 4, F1 = 7/2, F = 3, mF = 0, I1 = 1/2, I2 = 1/2, Omega = 1, P = None>

Branching ratios:
0.000185 to 
|g> =
+1.0000+0.0000j x |J = 3, F1 = 5/2, F = 2, mF = 0, I1 = 1/2, I2 = 1/2, Omega = 0, P = None>

0.001419 to 
|g> =
+0.9991+0.0000j x |J = 3, F1 = 5/2, F = 3, mF = 0, I1 = 1/2, I2 = 1/2, Omega = 0, P = None>
+0.0424+0.0000j x |J = 3, F1 = 7/2, F = 3, mF = 0, I1 = 1/2, I2 = 1/2, Omega = 0, P = None>

0.934721 to 
|g> =
+0.0424+0.0000j x |J = 3, F1 = 5/2, F = 3, mF = 0, I1 = 1/2, I2 = 1/2, Omega = 0, P = None>
-0.9991+0.0000j x |J = 3, F1 = 7/2, F = 3, mF = 0, I1 = 1/2, I2 = 1/2, Omega = 0, P = None>

0.032182 to 
|g> =
+1.0000+0.0000j x |J = 3, F1 = 7/2, F = 4, mF = 0, I1 =

**********
|e> =
+0.1887+0.0000j x |J = 3, F1 = 7/2, F = 3, mF = 0, I1 = 1/2, I2 = 1/2, Omega = -1, P = None>
+0.1887+0.0000j x |J = 3, F1 = 7/2, F = 3, mF = 0, I1 = 1/2, I2 = 1/2, Omega = 1, P = None>
+0.6815+0.0000j x |J = 4, F1 = 7/2, F = 3, mF = 0, I1 = 1/2, I2 = 1/2, Omega = -1, P = None>
-0.6815+0.0000j x |J = 4, F1 = 7/2, F = 3, mF = 0, I1 = 1/2, I2 = 1/2, Omega = 1, P = None>

Branching ratios:
0.000298 to 
|g> =
+0.9983+0.0000j x |J = 2, F1 = 3/2, F = 2, mF = 0, I1 = 1/2, I2 = 1/2, Omega = 0, P = None>
+0.0580+0.0000j x |J = 2, F1 = 5/2, F = 2, mF = 0, I1 = 1/2, I2 = 1/2, Omega = 0, P = None>

0.038548 to 
|g> =
+0.0580+0.0000j x |J = 2, F1 = 3/2, F = 2, mF = 0, I1 = 1/2, I2 = 1/2, Omega = 0, P = None>
-0.9983+0.0000j x |J = 2, F1 = 5/2, F = 2, mF = 0, I1 = 1/2, I2 = 1/2, Omega = 0, P = None>

0.001867 to 
|g> =
+1.0000+0.0000j x |J = 2, F1 = 5/2, F = 3, mF = 0, I1 = 1/2, I2 = 1/2, Omega = 0, P = None>

0.926590 to 
|g> =
+1.0000+0.0000j x |J = 4, F1 = 7/2, F = 3, mF = 0, I1 =

0.555556 to 
|g> =
+1.0000+0.0000j x |J = 3, F1 = 7/2, F = 4, mF = 0, I1 = 1/2, I2 = 1/2, Omega = 0, P = None>

0.000147 to 
|g> =
+1.0000+0.0000j x |J = 5, F1 = 9/2, F = 4, mF = 0, I1 = 1/2, I2 = 1/2, Omega = 0, P = None>

0.008330 to 
|g> =
+0.9996+0.0000j x |J = 5, F1 = 9/2, F = 5, mF = 0, I1 = 1/2, I2 = 1/2, Omega = 0, P = None>
+0.0274+0.0000j x |J = 5, F1 = 11/2, F = 5, mF = 0, I1 = 1/2, I2 = 1/2, Omega = 0, P = None>

0.006216 to 
|g> =
+0.0274+0.0000j x |J = 5, F1 = 9/2, F = 5, mF = 0, I1 = 1/2, I2 = 1/2, Omega = 0, P = None>
-0.9996+0.0000j x |J = 5, F1 = 11/2, F = 5, mF = 0, I1 = 1/2, I2 = 1/2, Omega = 0, P = None>

0.429752 to 
|g> =
+1.0000+0.0000j x |J = 5, F1 = 11/2, F = 6, mF = 0, I1 = 1/2, I2 = 1/2, Omega = 0, P = None>



In [14]:
def branching_ratio_table(excited_state, ground_states):
    nominal_state = excited_state.find_largest_component()
    nominal_J = nominal_state.J
    nominal_F1 = nominal_state.F1
    nominal_F = nominal_state.F
    mF = 0
    I1 = 1/2
    I2 = 1/2
    print(10*"*")
    print("|e> =")
    excited_state.print_state()
    print("\nBranching ratios:")
    
    branching_ratios = {-3:0, -2:0,-1:0, 0:0, 1:0, 2:0, 3:0}
    
    for ground_state in ground_states:
        J_g = ground_state.find_largest_component().J
        DeltaJ = J_g-nominal_J
        
        if np.abs(DeltaJ) < 4:
            br = np.real(calculate_branching_ratio(ground_state, excited_state))
            branching_ratios[DeltaJ] += br
            
    for DeltaJ, BR in branching_ratios.items():
        print("{:d}: {:.5f}".format(DeltaJ, BR))
        

In [15]:
for excited_state in excited_states:
    branching_ratio_table(excited_state,ground_states)

**********
|e> =
+0.7071+0.0000j x |J = 1, F1 = 1/2, F = 0, mF = 0, I1 = 1/2, I2 = 1/2, Omega = -1, P = None>
-0.7071+0.0000j x |J = 1, F1 = 1/2, F = 0, mF = 0, I1 = 1/2, I2 = 1/2, Omega = 1, P = None>

Branching ratios:
-3: 0.00000
-2: 0.00000
-1: 0.00000
0: 1.00000
1: 0.00000
2: 0.00000
3: 0.00000
**********
|e> =
+0.7071+0.0000j x |J = 1, F1 = 1/2, F = 0, mF = 0, I1 = 1/2, I2 = 1/2, Omega = -1, P = None>
+0.7071+0.0000j x |J = 1, F1 = 1/2, F = 0, mF = 0, I1 = 1/2, I2 = 1/2, Omega = 1, P = None>

Branching ratios:
-3: 0.00000
-2: 0.00000
-1: 0.66667
0: 0.00000
1: 0.33333
2: 0.00000
3: 0.00000
**********
|e> =
+0.7069+0.0000j x |J = 1, F1 = 1/2, F = 1, mF = 0, I1 = 1/2, I2 = 1/2, Omega = -1, P = None>
-0.7069+0.0000j x |J = 1, F1 = 1/2, F = 1, mF = 0, I1 = 1/2, I2 = 1/2, Omega = 1, P = None>

Branching ratios:
-3: 0.00000
-2: 0.00000
-1: 0.00000
0: 0.99987
1: 0.00000
2: 0.00013
3: 0.00000
**********
|e> =
-0.7069+0.0000j x |J = 1, F1 = 1/2, F = 1, mF = 0, I1 = 1/2, I2 = 1/2, Omega = -

-3: 0.00000
-2: 0.00000
-1: 0.00000
0: 0.96807
1: 0.00000
2: 0.03193
3: 0.00000
**********
|e> =
-0.2501+0.0000j x |J = 2, F1 = 5/2, F = 2, mF = 0, I1 = 1/2, I2 = 1/2, Omega = -1, P = None>
-0.2501+0.0000j x |J = 2, F1 = 5/2, F = 2, mF = 0, I1 = 1/2, I2 = 1/2, Omega = 1, P = None>
-0.6614+0.0000j x |J = 3, F1 = 5/2, F = 2, mF = 0, I1 = 1/2, I2 = 1/2, Omega = -1, P = None>
+0.6614+0.0000j x |J = 3, F1 = 5/2, F = 2, mF = 0, I1 = 1/2, I2 = 1/2, Omega = 1, P = None>

Branching ratios:
-3: 0.00000
-2: 0.07514
-1: 0.00000
0: 0.92486
1: 0.00000
2: 0.00000
3: 0.00000
**********
|e> =
+0.2496+0.0000j x |J = 2, F1 = 5/2, F = 2, mF = 0, I1 = 1/2, I2 = 1/2, Omega = -1, P = None>
-0.2496+0.0000j x |J = 2, F1 = 5/2, F = 2, mF = 0, I1 = 1/2, I2 = 1/2, Omega = 1, P = None>
+0.6615+0.0000j x |J = 3, F1 = 5/2, F = 2, mF = 0, I1 = 1/2, I2 = 1/2, Omega = -1, P = None>
+0.6615+0.0000j x |J = 3, F1 = 5/2, F = 2, mF = 0, I1 = 1/2, I2 = 1/2, Omega = 1, P = None>

Branching ratios:
-3: 0.00001
-2: 0.00000
-1: 

In [16]:
excited_state_approx = 1/np.sqrt(2)*(CoupledBasisState(J = 1, F1 = 3/2, F = 1, mF = 0, I1 = 1/2, I2 = 1/2, Omega = +1)
                                     + CoupledBasisState(J = 1, F1 = 3/2, F = 1, mF = 0, I1 = 1/2, I2 = 1/2, Omega = -1))
excited_state = find_closest_state(Hff_e,excited_state_approx,QN_e)
print("|e> =")
excited_state.print_state()
print("\nBranching ratios:")
#Loop over ground states
for ground_state in ground_states:
    br = np.real(calculate_branching_ratio(ground_state, excited_state))
    if br > 0.0001:
        print("{:5f} to \n|g> =".format(br))
        ground_state.print_state(tol = 0.01)
        print("")

|e> =
+0.6023+0.0000j x |J = 1, F1 = 3/2, F = 1, mF = 0, I1 = 1/2, I2 = 1/2, Omega = -1, P = None>
+0.6023+0.0000j x |J = 1, F1 = 3/2, F = 1, mF = 0, I1 = 1/2, I2 = 1/2, Omega = 1, P = None>
-0.3700+0.0000j x |J = 2, F1 = 3/2, F = 1, mF = 0, I1 = 1/2, I2 = 1/2, Omega = -1, P = None>
+0.3700+0.0000j x |J = 2, F1 = 3/2, F = 1, mF = 0, I1 = 1/2, I2 = 1/2, Omega = 1, P = None>

Branching ratios:
0.147371 to 
|g> =
-1.0000+0.0000j x |J = 0, F1 = 1/2, F = 1, mF = 0, I1 = 1/2, I2 = 1/2, Omega = 0, P = None>

0.336753 to 
|g> =
+1.0000+0.0000j x |J = 0, F1 = 1/2, F = 0, mF = 0, I1 = 1/2, I2 = 1/2, Omega = 0, P = None>

0.100769 to 
|g> =
+1.0000+0.0000j x |J = 2, F1 = 3/2, F = 1, mF = 0, I1 = 1/2, I2 = 1/2, Omega = 0, P = None>

0.026136 to 
|g> =
+0.9983+0.0000j x |J = 2, F1 = 3/2, F = 2, mF = 0, I1 = 1/2, I2 = 1/2, Omega = 0, P = None>
+0.0580+0.0000j x |J = 2, F1 = 5/2, F = 2, mF = 0, I1 = 1/2, I2 = 1/2, Omega = 0, P = None>

0.388970 to 
|g> =
+0.0580+0.0000j x |J = 2, F1 = 3/2, F = 2, mF 

In [19]:
np.sum(RMEs_states**2, axis = 0)

array([ 1.+0.j,  1.+0.j,  3.+0.j,  3.+0.j,  3.+0.j,  3.+0.j,  5.+0.j,
        5.+0.j,  5.+0.j,  5.+0.j,  7.+0.j,  7.+0.j,  3.+0.j,  3.+0.j,
        5.+0.j,  5.+0.j,  7.+0.j,  7.+0.j,  9.+0.j,  9.+0.j,  5.+0.j,
        5.+0.j,  7.+0.j,  7.+0.j,  7.+0.j,  7.+0.j,  9.+0.j,  9.+0.j,
        9.+0.j,  9.+0.j, 11.+0.j, 11.+0.j])